# Análisis Exploratorio de Datos
## Sistema de Monitoreo para Personas Sordas

In [ ]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from database_connection import execute_query

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

%matplotlib inline

## 1. Cargar Datos

In [ ]:
with open('../data/02_extract_features.sql', 'r') as f:
    query = f.read()

data = execute_query(query)
df = pd.DataFrame(data)

print(f"Datos cargados: {df.shape[0]} usuarios, {df.shape[1]} features")
df.head()

## 2. Inspección Básica

In [ ]:
print("Información del DataFrame:")
df.info()

print("\nValores nulos:")
print(df.isnull().sum())

print("\nEstadísticas descriptivas:")
df.describe()

## 3. Distribución de Features

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('user_id')

n_cols = 3
n_rows = int(np.ceil(len(numeric_cols) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(col, fontsize=10)
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label='Media')
    axes[idx].axvline(df[col].median(), color='green', linestyle='--', linewidth=2, label='Mediana')
    axes[idx].legend()

for idx in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('../output/feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Matriz de Correlación

In [ ]:
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación de Features', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('../output/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nCorrelaciones más fuertes (|r| > 0.5):")
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

corr_df = pd.DataFrame(corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlación'])
corr_df = corr_df.sort_values('Correlación', key=abs, ascending=False)
print(corr_df)

## 5. Detección de Outliers

In [ ]:
n_cols = 3
n_rows = int(np.ceil(len(numeric_cols) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].boxplot(df[col].dropna(), vert=True)
    axes[idx].set_title(col, fontsize=10)
    axes[idx].set_ylabel('Valor')
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
    axes[idx].text(0.5, 0.95, f'Outliers: {len(outliers)}', 
                   transform=axes[idx].transAxes, ha='center', va='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

for idx in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('../output/outliers_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Análisis de Segmentos Naturales

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Completion Rate vs Frustración', 
                    'Atención Visual vs Fatiga Visual',
                    'Distracción vs Pausas',
                    'Engagement vs Intervenciones')
)

fig.add_trace(
    go.Scatter(x=df['completion_rate'], y=df['avg_frustration'], 
               mode='markers', name='Usuarios',
               marker=dict(size=8, opacity=0.6)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df['avg_visual_attention'], y=df['avg_visual_fatigue'], 
               mode='markers', name='Usuarios',
               marker=dict(size=8, opacity=0.6)),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df['distraction_events_per_hour'], y=df['avg_pause_count'], 
               mode='markers', name='Usuarios',
               marker=dict(size=8, opacity=0.6)),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df['avg_engagement_score'], y=df['intervention_count_per_activity'], 
               mode='markers', name='Usuarios',
               marker=dict(size=8, opacity=0.6)),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="Relaciones entre Features Clave")
fig.write_html('../output/scatter_relationships.html')
fig.show()

## 7. Preparación de Features para Clustering

In [ ]:
features_for_clustering = [
    'completion_rate',
    'avg_frustration',
    'avg_visual_attention',
    'avg_visual_fatigue',
    'distraction_events_per_hour',
    'drowsiness_events_per_hour',
    'avg_pause_count',
    'intervention_count_per_activity',
    'avg_engagement_score',
    'preference_easy_activities'
]

df_clustering = df[['user_id'] + features_for_clustering].copy()

print(f"Features seleccionadas para clustering: {len(features_for_clustering)}")
print("\nEstadísticas de features seleccionadas:")
print(df_clustering[features_for_clustering].describe())

df_clustering.to_csv('../output/features_for_clustering.csv', index=False)
print("\nFeatures guardadas en: output/features_for_clustering.csv")

## 8. Conclusiones del EDA

In [ ]:
print("CONCLUSIONES DEL ANÁLISIS EXPLORATORIO:")
print("="*60)

print(f"\n1. DATOS GENERALES:")
print(f"   - Total usuarios analizados: {len(df)}")
print(f"   - Features totales: {len(numeric_cols)}")
print(f"   - Features para clustering: {len(features_for_clustering)}")

print(f"\n2. COMPLETION RATE:")
print(f"   - Promedio: {df['completion_rate'].mean():.2%}")
print(f"   - Mediana: {df['completion_rate'].median():.2%}")
print(f"   - Desviación estándar: {df['completion_rate'].std():.2%}")

print(f"\n3. FRUSTRACIÓN:")
print(f"   - Promedio: {df['avg_frustration'].mean():.3f}")
print(f"   - Usuarios con alta frustración (>0.7): {(df['avg_frustration'] > 0.7).sum()}")

print(f"\n4. ATENCIÓN VISUAL:")
print(f"   - Promedio: {df['avg_visual_attention'].mean():.1f}%")
print(f"   - Usuarios con baja atención (<60%): {(df['avg_visual_attention'] < 60).sum()}")

print(f"\n5. DISTRACCIÓN:")
print(f"   - Eventos promedio por hora: {df['distraction_events_per_hour'].mean():.2f}")
print(f"   - Usuarios muy distraídos (>8/hora): {(df['distraction_events_per_hour'] > 8).sum()}")

print(f"\n6. PAUSAS:")
print(f"   - Promedio por actividad: {df['avg_pause_count'].mean():.2f}")
print(f"   - Usuarios con muchas pausas (>6): {(df['avg_pause_count'] > 6).sum()}")

print(f"\n7. INTERVENCIONES:")
print(f"   - Promedio por actividad: {df['intervention_count_per_activity'].mean():.2f}")
print(f"   - Respuesta a video: {df['response_to_video'].mean():.2%}")
print(f"   - Respuesta a texto: {df['response_to_text'].mean():.2%}")
print(f"   - Respuesta a vibración: {df['response_to_vibration'].mean():.2%}")

print("\n" + "="*60)